<div><img src="https://radar.community.uaf.edu/wp-content/uploads/sites/667/2021/03/HydroSARbanner.jpg" width="100%" /></div>

**NASA A.37 Project:** Integrating SAR Data for Improved Resilience and Response to Weather-Related Disasters
**PI:** Franz J. Meyer

# HydroSAR Transition workshop

### Creating HYDRO30 and FD30 products

This notebook mimics the workflow the HyP3 `WATER_MAP` job will follow to create an On Demand HydroSAR HYDRO30 and FD30 products. It is intended demonstrate *how* HyP3 goes from an On Demand request to a product, as well as provide a platform for testing/tuning/refinement of the water products.

Step 1 is a required data marshaling step that will be handled transparently inside the container. Importantly, steps 2 and 3 become the plugin's primary interface, and the options provided to the `make_water_map` and `make_flood_map` functions are able to be exposed to users.

## 1. Generate input dual-pol RTC product

Water extent mapping requires Radiometrically Terrain Corrected (RTC) Sentinel-1 scenes in order to accurately connect backscatter features to the input HAND data. The next few cells will use HyP3's Python SDK to request an RTC product from HyP3 for the Sentinel-1 granule (scene):

[S1A_IW_SLC__1SDV_20220723T000409_20220723T000436_044220_05471E_D5FE](https://search.asf.alaska.edu/#/?searchType=List%20Search&searchList=S1A_IW_SLC__1SDV_20220723T000409_20220723T000436_044220_05471E_D5FE&resultsLoaded=true&granule=S1A_IW_SLC__1SDV_20220723T000409_20220723T000436_044220_05471E_D5FE-SLC&zoom=5.094&center=90.113,15.694)

### 1.1 Connect to HyP3

In [ ]:
import hyp3_sdk as sdk

hyp3 = sdk.HyP3(prompt=True)

### 1.2  Submit an On Demand request for the input RTC product

Currently, the RTC options provided to `hyp3.submit_rtc_job` as specified are expected/required by `make_water_map` and are not recommended to be changed, except for:

* `speckle_filter` -- Apply an Enhanced Lee speckle filter.

* `dem_matching` -- Coregisters SAR data to the DEM, rather than using dead reckoning based on orbit files. See the [Terrain Correction](https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/#terrain-correction) in the RTC Product Guide for more information.

* `include_dem` -- include a GeoTIF of the Copernicus DEM pixel aligned to the RTC products. This GeoTIFF isn't used by `make_water_map` but was used to create the RTC product and the HAND data

* `include_rgb` -- include a false-color RGB decomposition for dual-pol granules. This GeoTIFF isn't used by `make_water_map` but may help you [interpret the RTC image](https://github.com/ASFHyP3/hyp3-lib/blob/develop/docs/rgb_decomposition.md#interpreting-the-images) and evaluate the water extent map


In [ ]:
granule = 'S1A_IW_SLC__1SDV_20220723T000409_20220723T000436_044220_05471E_D5FE'

job = hyp3.submit_rtc_job(
    granule, name='water-extent-example',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)

print(job)

In [ ]:
job = hyp3.watch(job)

### 1.4 Download the RTC product

RTC products are downloaded as Zip files and then extracted. The GeoTIFF for the VV and VH polarizations are selected from the (unzipped) product folder.

In [ ]:
from hyp3_sdk.util import extract_zipped_product

product_zip = job.download_files()[0]
product_folder = extract_zipped_product(product_zip)

vv_raster = product_folder / f'{product_zip.stem}_VV.tif'
vh_raster = product_folder / f'{product_zip.stem}_VH.tif'

print(f'VV raster: {vv_raster}')
print(f'VH raster: {vv_raster}')

### Interlude: A bit about the ASF Tools Python Package

HyP3 uses the [ASF Tools Python Package](https://hyp3-docs.asf.alaska.edu/tools/asf_tools/) to create the HydroSAR products. Primarily there are two main "science" interfaces:
* The `make_water_map` function in the `asf_tools.water_map` module, which creates the HYDRO30 product
* The `make_flood_map` function in the `asf_tools.flood_map` module, which creates the FD30 product

They *should* include all the necessary options for tuning the product -- if it doesn't, and you'd like other things exposed, either:
* Ask us to expose it by:
  * [Chatting with us in Gitter](https://gitter.im/ASFHyP3/community)
  * [Opening an issue](https://github.com/ASFHyP3/asf-tools/issues/new/choose)

* [Fork](https://github.com/ASFHyP3/hyp3-sdk/fork) and propose changes in a Pull Request. See our [contributing guidelines](https://hyp3-docs.asf.alaska.edu/contributing/).

For a detailed description for arguments used in the `make_water_map` function, run the cell below:


In [ ]:
from asf_tools.water_map import make_water_map

?make_water_map

For a detailed description of the arguments to the `make_flood_map` function, run the cell below:

In [ ]:
from asf_tools.flood_map import make_flood_map

?make_flood_map

### 2. Creating the HYDRO30 product

Once we have marshalled all the required input data, we can generate the HYDRO30 product!

*Note:* this will take $\approx$ 30 minutes to complete.

In [ ]:
from asf_tools.water_map import make_water_map

water_map = vv_raster.parent / vv_raster.name.replace('_VV.tif', '_WM.tif')

make_water_map(water_map, vv_raster, vh_raster, tile_shape=(100, 100),
               max_vv_threshold=-15.5, max_vh_threshold=-23., hand_threshold=15., hand_fraction=0.8)

print(f'HYDRO30 surface water extent: {water_map}')

### 3. Creating the FD30 product

We are now able to estimate flood depths for our AOI with the VV RTC raster from our RTC job, HYDRO30 surface water extent map, and HAND raster produced while creating the surface water extent map.

**Note**: this will take $\approx$4-5 hours to complete when using the iterative method.

In [ ]:
flood_map = water_map.parent / water_map.name.replace('_WM.tif', '_FD.tif')
hand_raster = water_map.parent / water_map.name.replace('_WM.tif', '_WM_HAND.tif')
print(f'HAND raster: {hand_raster}')


make_water_map(flood_map, vv_raster, water_map, hand_raster,
               estimator='iterative', water_level_sigma=3.,
               known_water_threshold=30., iterative_bounds=(0, 15))

print(f'FD30 flood_map: {flood_map}')

## Operations

To do all this operationally with HyP3, we add ASF Tools to the HyP3 GAMMA plugin and create a `water_map` entrypoint, which
* Radiometerically Terrain Corrects the input scene
* Runs `make_water_map` to make a HYDRO30 product using the RTC ouptuts
* Runs `make_flood_map` to make a FD30 product using the HYDRO30 and the RTC outputs

The relevant parts of the [HyP3 GAMMA plugin](https://github.com/ASFHyP3/hyp3-gamma) are:
* Docker image build steps: <https://github.com/ASFHyP3/hyp3-gamma/blob/develop/Dockerfile>
* `water_map` entrypoint: <https://github.com/ASFHyP3/hyp3-gamma/blob/develop/hyp3_gamma/__main__.py#L172>
